In [1]:
!pip install langchain datasets transformers sentence-transformers faiss-cpu evaluate rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.6 MB/s eta 0:00

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [3]:
from datasets import load_dataset

# Load a subset of SQuAD
dataset = load_dataset("squad")

contexts = dataset["train"]["context"] + dataset["validation"]["context"]

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
from langchain.schema import Document #LangChain document wrappers
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs = [Document(page_content=ctx, metadata={"source": f"squad_{i}"})
        for i, ctx in enumerate(set(contexts))]  # Remove duplicates
splitter    = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs  = splitter.split_documents(docs)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS # Facebook AI Similarity Search (for semantic search)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(split_docs, embedding_model)
retriever = db.as_retriever(search_kwargs={"k": 3})

/tmp/ipykernel_35/2432356392.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
2025-05-19 01:04:43.922907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747616684.374840      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747616684.485526      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer


MODEL_ID = "google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
)

qa_pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    truncation = True,
)

llm = HuggingFacePipeline(pipeline=qa_pipe)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipykernel_35/1407868154.py:23: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=qa_pipe)


In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

ZERO_PROMPT = PromptTemplate(
    template=(
        "Context:\n{context}\n\n"
        "Question: {question}\n"
        "Answer:"
    ),
    input_variables=["context", "question"],
)

COT_PROMPT = PromptTemplate(
    template=(
        "Use only the context below to answer.\n\n"
        "First think step-by-step, then answer in one short sentence.\n\n"
        "Context:\n{context}\n\n"
        "Question: {question}\n"
        "Answer:"
    ),
    input_variables=["context", "question"],
)

def make_rag_chain(prompt: PromptTemplate):
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
    )

rag_zero = make_rag_chain(ZERO_PROMPT)
rag_cot  = make_rag_chain(COT_PROMPT)


In [25]:
from datasets import load_dataset, DatasetDict
import evaluate

dataset = load_dataset("squad")
test_subset = dataset["validation"].shuffle(seed=42).select(range(2000))
data = DatasetDict({"test": test_subset})

In [13]:
for ex in data["test"].select(range(10)):
    q     = ex["question"]
    ans1   = rag_zero.run(q).strip()
    ans2 = rag_cot.run(q).strip()
    golds = ex["answers"]["text"]  # this is a list of gold answers

    print(f"Q:  {q}")
    print(f"Z:  {ans1}")
    print(f"C:  {ans2}")
    # join multiple gold answers with semicolons, or show “No answer” if empty
    print("GT: " + ("; ".join(golds) if golds else "No answer"))
    print("-" * 40)

/tmp/ipykernel_35/1007999310.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ans1   = rag_zero.run(q).strip()


Q:  In what year did Massachusetts first require children to be educated in schools?
Z:  1852
C:  1852
GT: 1852; 1852; 1852
----------------------------------------
Q:  When were stromules discovered?
Z:  1962
C:  1962
GT: 1962; 1962; 1962
----------------------------------------
Q:  Which artist who had a major influence on the Gothic Revival is represented in the V&A's British galleries?
Z:  unanswerable
C:  John Ruskin
GT: Horace Walpole; Horace Walpole; Horace Walpole
----------------------------------------
Q:  In 1890, who did the university decide to team up with?
Z:  several regional colleges and universities
C:  several regional colleges and universities
GT: several regional colleges and universities; Des Moines College, Kalamazoo College, Butler University, and Stetson University; Des Moines College, Kalamazoo College, Butler University, and Stetson University; Des Moines College, Kalamazoo College, Butler University, and Stetson University
-----------------------------------

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Q:  Who got a touchdown making the score 10-7?
Z:  Anderson
C:  Anderson
GT: Jonathan Stewart; Jonathan Stewart; Stewart
----------------------------------------
Q:  How many Examination Boards exist in India?
Z:  multiple states
C:  multiple states
GT: 30; 30; 30
----------------------------------------
Q:  Who started rumors in 2008 that ABC would sell its ten owned-and-operated stations?
Z:  Caris & Co
C:  Caris & Co
GT: Caris & Co.; Caris & Co.; Caris & Co.
----------------------------------------
Q:  Which network broadcasted the 50th Super Bowl game? 
Z:  CBS
C:  CBS
GT: CBS; CBS; CBS
----------------------------------------
Q:  Why was this short termed organization created?
Z:  organizations that didn't fit anywhere else
C:  It is not designated specifically for charitable organizations or any specific organizational or tax-law status, however; it encompasses anything that is not classifiable as another category.
GT: coordinate the response to the embargo; to coordinate the res

In [26]:
import string

def normalize_text(s: str) -> str:
    # lowercase, strip punctuation, collapse whitespace
    s = s.lower().translate(str.maketrans("", "", string.punctuation))
    return " ".join(s.split())

def print_chain_metrics(chain, data):
    preds, refs = [], []
    rouge_hyp, rouge_ref, bleu_refs = [], [], []

    total = len(data["test"])
    for idx, ex in enumerate(data["test"], 1):
        # get model output
        raw = chain({"query": ex["question"]})["result"]
        text = raw.strip()

        # SQuAD pred/ref
        preds.append({"id": ex["id"], "prediction_text": text})
        refs.append({
            "id": ex["id"],
            "answers": {
                "text": ex["answers"]["text"],
                "answer_start": ex["answers"]["answer_start"]
            }
        })

        # normalize for ROUGE/BLEU
        pred_norm = normalize_text(text)

        # take first gold span (or empty)
        golds = ex["answers"]["text"]
        first_gold = golds[0] if golds else ""
        gold_norm = normalize_text(first_gold)

        rouge_hyp.append(pred_norm)
        rouge_ref.append(gold_norm)
        # BLEU needs a list of all normalized refs
        bleu_refs.append([normalize_text(g) for g in golds] or [""])

        print(f"[{idx}/{total}] Done.\n")

    print("All examples processed — now computing metrics…\n")

    squad  = evaluate.load("squad")
    rouge  = evaluate.load("rouge")
    bleu   = evaluate.load("bleu")
    s_res = squad.compute(predictions=preds, references=refs)
    r_res = rouge.compute(predictions=rouge_hyp, references=rouge_ref)
    b_res = bleu.compute(predictions=rouge_hyp, references=bleu_refs)

    print(f"F1:           {s_res['f1']:.2f}")
    print(f"ROUGE-L F1:   {r_res['rougeL'] * 100:.2f}")
    print(f"BLEU:         {b_res['bleu'] * 100:.2f}")

print_chain_metrics(rag_zero, data)

[1/2000] Done.

[2/2000] Done.

[3/2000] Done.

[4/2000] Done.

[5/2000] Done.

[6/2000] Done.

[7/2000] Done.

[8/2000] Done.

[9/2000] Done.

[10/2000] Done.

[11/2000] Done.

[12/2000] Done.

[13/2000] Done.

[14/2000] Done.

[15/2000] Done.

[16/2000] Done.

[17/2000] Done.

[18/2000] Done.

[19/2000] Done.

[20/2000] Done.

[21/2000] Done.

[22/2000] Done.

[23/2000] Done.

[24/2000] Done.

[25/2000] Done.

[26/2000] Done.

[27/2000] Done.

[28/2000] Done.

[29/2000] Done.

[30/2000] Done.

[31/2000] Done.

[32/2000] Done.

[33/2000] Done.

[34/2000] Done.

[35/2000] Done.

[36/2000] Done.

[37/2000] Done.

[38/2000] Done.

[39/2000] Done.

[40/2000] Done.

[41/2000] Done.

[42/2000] Done.

[43/2000] Done.

[44/2000] Done.

[45/2000] Done.

[46/2000] Done.

[47/2000] Done.

[48/2000] Done.

[49/2000] Done.

[50/2000] Done.

[51/2000] Done.

[52/2000] Done.

[53/2000] Done.

[54/2000] Done.

[55/2000] Done.

[56/2000] Done.

[57/2000] Done.

[58/2000] Done.

[59/2000] Done.

[60/20

In [28]:
print_chain_metrics(rag_cot, data)

[1/2000] Done.

[2/2000] Done.

[3/2000] Done.

[4/2000] Done.

[5/2000] Done.

[6/2000] Done.

[7/2000] Done.

[8/2000] Done.

[9/2000] Done.

[10/2000] Done.

[11/2000] Done.

[12/2000] Done.

[13/2000] Done.

[14/2000] Done.

[15/2000] Done.

[16/2000] Done.

[17/2000] Done.

[18/2000] Done.

[19/2000] Done.

[20/2000] Done.

[21/2000] Done.

[22/2000] Done.

[23/2000] Done.

[24/2000] Done.

[25/2000] Done.

[26/2000] Done.

[27/2000] Done.

[28/2000] Done.

[29/2000] Done.

[30/2000] Done.

[31/2000] Done.

[32/2000] Done.

[33/2000] Done.

[34/2000] Done.

[35/2000] Done.

[36/2000] Done.

[37/2000] Done.

[38/2000] Done.

[39/2000] Done.

[40/2000] Done.

[41/2000] Done.

[42/2000] Done.

[43/2000] Done.

[44/2000] Done.

[45/2000] Done.

[46/2000] Done.

[47/2000] Done.

[48/2000] Done.

[49/2000] Done.

[50/2000] Done.

[51/2000] Done.

[52/2000] Done.

[53/2000] Done.

[54/2000] Done.

[55/2000] Done.

[56/2000] Done.

[57/2000] Done.

[58/2000] Done.

[59/2000] Done.

[60/20

In [36]:
def chat_cli(chain):
    """
    Simple command-line chat: type a question, see the bot’s answer.
    Type exit or quit to stop.
    """
    print("🤖 Chatbot ready! Type your question (or 'exit' to quit).")
    while True:
        q = input("You: ").strip()
        if q.lower() in ("exit", "quit"):
            print("Goodbye! 👋")
            break

        # run the chain
        a = chain.run(q).strip()
        print("Bot:", a, "\n")

# to start chatting:
chat_cli(rag_zero)


🤖 Chatbot ready! Type your question (or 'exit' to quit).


You:  What is the capital of France


Bot: Paris 



You:  How does photosynthesis work?


Bot: light is transformed into chemical energy 



You:  Summarize the plot of Romeo and Juliet in two sentences


Bot: The plot of Romeo and Juliet is a fictionalized account of the life of two young lovers. 



You:  quit


Goodbye! 👋


In [37]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# 1) memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 2) build conversational RAG chain (CoT)
conv_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": ZERO_PROMPT}
)


# 3) a CLI loop that uses the conv_chain
def chat_cli_with_memory(chain):
    print("🤖 Chatbot (with memory) ready! Type a question, or 'exit' to quit.")
    while True:
        q = input("You: ").strip()
        if q.lower() in ("exit", "quit"):
            print("Goodbye! 👋")
            break

        # run the conversational chain
        result = chain({"question": q})
        answer = result["answer"].strip()

        print("Bot:", answer, "\n")

# start it
chat_cli_with_memory(conv_chain)

🤖 Chatbot (with memory) ready! Type a question, or 'exit' to quit.


You:  Summarize the plot of Romeo and Juliet in two sentences


Bot: The plot of Romeo and Juliet is a fictionalized account of the life of two young lovers. 



You:  How does photosynthesis work?


Bot: light is transformed into chemical energy 



...

Bot: vanilla 



You:  Tell me one fact about the Eiffel Tower.


Bot: tallest buildings in the European Union 



You:  quit


Goodbye! 👋


In [38]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# remember only the last 2 exchanges
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    k=2   # <-- only keep the last 2 messages in memory
)

conv_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": ZERO_PROMPT}
)

# 3) a CLI loop that uses the conv_chain
def chat_cli_with_memory(chain):
    print("🤖 Chatbot (with memory) ready! Type a question, or 'exit' to quit.")
    while True:
        q = input("You: ").strip()
        if q.lower() in ("exit", "quit"):
            print("Goodbye! 👋")
            break

        # run the conversational chain
        result = chain({"question": q})
        answer = result["answer"].strip()

        print("Bot:", answer, "\n")

# start it
chat_cli_with_memory(conv_chain)

🤖 Chatbot (with memory) ready! Type a question, or 'exit' to quit.


You:  Summarize the plot of Romeo and Juliet in two sentences


Bot: The plot of Romeo and Juliet is a fictionalized account of the life of two young lovers. 



You:  How does photosynthesis work?


Bot: light is transformed into chemical energy 



...

Bot: vanilla 



You:  Tell me one fact about the Eiffel Tower.


Bot: tallest buildings in the European Union 



You:  quit


Goodbye! 👋
